### Install and import require packages

In [0]:
import os, time
%matplotlib inline
import matplotlib.pyplot as plt
import itertools
import pickle
import imageio

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torchvision import datasets, transforms

### Set number of epochs, learning rate and batch size

In [0]:
batch_size = 128
lr = 0.0002
num_of_epochs = 20

### Load Dataset

In [0]:
img_size = 64
transform = transforms.Compose([
        transforms.Scale(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True)

### Model

In [0]:
class ImageDiscriminator(nn.Module):
    def __init__(self, noise_vector_size=128):
        super(ImageDiscriminator, self).__init__()
        ##TODO: implement initial 4 Convolution layers using noise_vector_size with linear kernel(=4), stride(=2) and padding(=1)##
        ##TODO: add BatchNorm layers from second conv layer till fourth conv layer##
        self.conv5 = nn.Conv2d(noise_vector_size * 8, 1, 4, 1, 0)

    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    def forward(self, input):
        ##TODO: implement forward pass (be cautious when adding BatchNorm layers)##
        x = F.sigmoid(self.conv5(x))

        return x

In [0]:
class ImageGenerator(nn.Module):
    def __init__(self, noise_vector_size=128):
        super(ImageGenerator, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(100, noise_vector_size * 8, 4, 1, 0)
        ##TODO: implement rest 4 Tranpose Convolution layers using noise_vector_size with linear kernel(=4), stride(=2) and padding(=1)##
        ##TODO: add BatchNorm layers from first conv layer till fourth conv layer##

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        ##TODO: implement forward pass (be cautious when adding BatchNorm layers)##
        x = F.tanh(self.deconv5(x))

        return x

In [0]:
###TODO: Replace normal weight initialization with xavier intialization##
def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()


In [0]:
fixed_noise = torch.randn((5 * 5, 100)).view(-1, 100, 1, 1)
fixed_noise = Variable(fixed_noise.cuda(), volatile=True)

In [0]:
def show_epoch_result(num_epoch, show = False, save = False, path = 'result.png', isFix=False):
    random_noise = torch.randn((5*5, 100)).view(-1, 100, 1, 1)
    random_noise = Variable(random_noise.cuda(), volatile=True)

    G.eval()
    if isFix:
        test_images = G(fixed_noise)
    else:
        test_images = G(random_noise)
    G.train()

    size_figure_grid = 5
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for k in range(5*5):
        i = k // 5
        j = k % 5
        ax[i, j].cla()
        ax[i, j].imshow(test_images[k, 0].cpu().data.numpy(), cmap='gray')

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')
    plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()


def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    ##TODO: implement show_train_hist function to plot losses histograms ##

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

In [0]:
G = ImageGenerator(128)
D = ImageDiscriminator(128)
G.weight_init(mean=0.0, std=0.02)
D.weight_init(mean=0.0, std=0.02)
G.cuda()
D.cuda()

### Optimization

In [0]:
BCE_loss = nn.BCELoss()
G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
D_optimizer = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))


### Results folder to save visualizations

In [0]:
if not os.path.isdir('results'):
    os.mkdir('results')
if not os.path.isdir('results/dcgan'):
    os.mkdir('results/dcgan')
if not os.path.isdir('results/dcgan/Random_results'):
    os.mkdir('results/dcgan/Random_results')
if not os.path.isdir('results/dcgan/Fixed_results'):
    os.mkdir('results/dcgan/Fixed_results')

train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_times'] = []
train_hist['total_time'] = []
num_iter = 0

### Training

In [0]:
print('Started training...')
start_time = time.time()
for epoch in range(num_of_epochs):
    D_losses = []
    G_losses = []
    epoch_start_time = time.time()
    for x_, _ in train_loader:
        ##TODO: fill-in training discriminator procedure (hint: similar to mlp_gan.ipynb implementation) ##
        D_losses.append(D_train_loss.item())

        ##TODO: fill-in training generator procedure (hint: similar to mlp_gan.ipynb implementation) ##
        G_losses.append(G_train_loss.item())

        num_iter += 1

    epoch_end_time = time.time()
    per_epoch_time = epoch_end_time - epoch_start_time


    print('[%d/%d] - time: %.2f, loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), num_of_epochs, per_epoch_time, torch.mean(torch.FloatTensor(D_losses)),
                                                              torch.mean(torch.FloatTensor(G_losses))))
    p = 'results/dcgan/Random_results/' + str(epoch + 1) + '.png'
    fixed_p = 'results/dcgan/Fixed_results/' + str(epoch + 1) + '.png'
    
    ##TODO: save the epoch results with fixed_noise and randon_noise ##
    
    train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
    train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))
    train_hist['per_epoch_times'].append(per_epoch_time)

end_time = time.time()
total_time = end_time - start_time
train_hist['total_time'].append(total_time)

print("Avg per epoch time: %.2f, total %d epochs time: %.2f" % (torch.mean(torch.FloatTensor(train_hist['per_epoch_times'])), num_of_epochs, total_time))
print("Training finish!... save training results")

##TODO: save the generator and discriminator weights into .pkl files ##

with open('results/dcgan/train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

##TODO: save training histograms ##